Transformersで扱うモデルは基本的に同時に複数の文章(リスト)を入力として扱う。そのため、入力として期待されているのは２次元のtensorである。

上記の理由から、下記のコードは1次元のtensorを入力していることから、Errorを返す。

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import pprint

In [2]:
# 事前学習済のトークナイザとモデルを読み込む
ckpt = 'distilbert-base-uncased-finetuned-sst-2-english'
tokenizer = AutoTokenizer.from_pretrained(ckpt)
model = AutoModelForSequenceClassification.from_pretrained(ckpt)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [3]:
# 単一の文章をそのままid列に変換
sentence = 'Saito is so popular with girls.'
tokens = tokenizer.tokenize(sentence)
ids = tokenizer.convert_tokens_to_ids(tokens)
bad_input_ids = torch.tensor(ids)

pprint.pprint(bad_input_ids)
print(f'Dimensions: {bad_input_ids.ndim}')

tensor([18952,  3406,  2003,  2061,  2759,  2007,  3057,  1012])
Dimensions: 1


In [4]:
# ここでErrorになる。
model(bad_input_ids)

IndexError: too many indices for tensor of dimension 1

このErrorはmodelへの入力を期待されている2次元のtensorへ変換することで解消できる。

In [5]:
# id列を2次元のtensorに変換
input_ids = torch.tensor([ids])

pprint.pprint(input_ids)
print(f'Dimensions: {input_ids.ndim}')

tensor([[18952,  3406,  2003,  2061,  2759,  2007,  3057,  1012]])
Dimensions: 2


In [6]:
# unsqueeze()を使っても次元拡張ができる
## https://pytorch.org/docs/stable/generated/torch.unsqueeze.html#torch.unsqueeze
input_ids = bad_input_ids.unsqueeze(0)

pprint.pprint(input_ids)
print(f'Dimensions: {input_ids.ndim}')

tensor([[18952,  3406,  2003,  2061,  2759,  2007,  3057,  1012]])
Dimensions: 2


In [7]:
out = model(input_ids).logits
out

tensor([[-2.1485,  2.1525]], grad_fn=<AddmmBackward0>)

In [8]:
import torch.nn.functional as F

F.softmax(out, dim=-1)

tensor([[0.0134, 0.9866]], grad_fn=<SoftmaxBackward0>)

また、tokenizerそのものを使った時の出力も確認してみると、以下のようになる。

In [9]:
tokenized_input = tokenizer(sentence, return_tensors='pt')
tokenized_input['input_ids']

tensor([[  101, 18952,  3406,  2003,  2061,  2759,  2007,  3057,  1012,   102]])

これは先ほどのinput_idsと同じ次元をもつtensorを返し、かつ初頭と終端に`[CLS]`と`[SEP]`という特殊トークンを挿入している事に注意する。

続いて、入力を複数の文章(リスト)としてmodelに入力するまでの処理を考える。
仮にトークナイズ・id化済みのid列が下記のように得られたとする。



In [10]:
batched_ids = [
    [100],
    [200, 200],
    [300, 300, 300],
]

これをそのままtensorに変換しようとすると下記のようにエラーになる。
これはtensorはリストの各要素(文章)の長さが等しい事を前提としている事が理由である。

In [11]:
torch.tensor(batched_ids)

ValueError: expected sequence of length 1 at dim 1 (got 2)

そこで、複数の文章をmodelに入力するには工夫をする必要があり、その一つがpaddingである。

paddingとは、特殊トークンを使ってid列の長さをそろえることである。

In [12]:
padding_id = tokenizer.pad_token_id

batched_ids = [
    [200, padding_id, padding_id],
    [200, 200, padding_id],
    [200, 200, 200],
]

torch.tensor(batched_ids)

tensor([[200,   0,   0],
        [200, 200,   0],
        [200, 200, 200]])

ちなみに、上記の例の場合、`padding_id`は`[PAD]`というトークンを表している。

In [14]:
tokenizer.decode(batched_ids[0])

'[unused195] [PAD] [PAD]'

続いて、このid列を入力としたときの出力を確認する。

In [15]:
no_pad_tensor = torch.tensor([[200, 200]])
pad_tensor = torch.tensor([[200, 200, padding_id]])

batched_ids_tensor = torch.tensor(batched_ids)

# paddingなし/ありで出力は変わって欲しく無い。
pprint.pprint(model(no_pad_tensor).logits)  # paddingなし
pprint.pprint(model(pad_tensor).logits)      # paddingあり

pprint.pprint(model(batched_ids_tensor).logits)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.3374, -1.2163]], grad_fn=<AddmmBackward0>)
tensor([[ 1.0182, -0.9527],
        [ 1.3374, -1.2163],
        [ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)


このとき、[PAD]トークンの有無によって出力が異なると実用上好ましくない(文意が一緒なのに、出力が異なるため)が、上の例では、paddingの有無によって出力が異なってしまっている。

そこで、tokenizerでは通常自動でattention_maskを付加する事で、[PAD]トークンをAttention機構の計算過程で無視するという仕組みを実現している。

同様の処理を行うため、先ほどのbatched_idsでpadding_idを挿入した位置に着目してattention_maskを作る。

In [16]:
attention_mask = torch.tensor(
    [
        [1, 0, 0],
        [1, 1, 0],
        [1, 1, 1]
    ]
)

outputs = model(batched_ids_tensor, attention_mask=attention_mask)

pprint.pprint(outputs.logits)

tensor([[-0.6815,  0.9039],
        [ 0.5803, -0.4125],
        [ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)


ここで注目すべきは、attention_maskを入力した事でoutput.logits[1]とmodel(no_pad_tensor).logitsが一致する事である。

これらの処理を踏まえて、tokenizerそのもので文章のリストを変換した時の出力を確認すると、以下のようになる。

In [24]:
sentences = [
    'I love you.',
    'I love you, too.',
    'I am loving hamburger and fries.',
]

pprint.pprint(tokenizer(sentences,  padding='longest', return_tensors='pt'))

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 'input_ids': tensor([[  101,  1045,  2293,  2017,  1012,   102,     0,     0,     0],
        [  101,  1045,  2293,  2017,  1010,  2205,  1012,   102,     0],
        [  101,  1045,  2572,  8295, 24575,  1998, 22201,  1012,   102]])}


この処理で確認できるように、tokenizerによって文章のリストを変換した際には、paddingによってリスト長の揃ったid列と同時に、[PAD]トークンの位置に０を持つattention_maskを出力している事がわかる。
この時、paddingは`longest`と指定している事に注意する。

最後に、長い文章を入力とする事を考える。
通常のhuggingfaceのBertモデルは最大token長512または１０２４としている事が多く、これを超える入力を与えるとエラーとなる。

そこで、より長い文章を扱うには、


1.   長い文章を入力可能なモデルを使う
2.   入力文を切り詰める(truncation)

のいずれかを選択する必要がある。
tokenizerで選択肢２を行うためには、引数の`truncation=True`を指定する事で実現できる。